In [ ]:
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.neural_network import MLPClassifier
import pandas, numpy, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
from sklearn import model_selection, svm
import pandas as pd
from keras.utils import np_utils
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional


In [ ]:
trainDF = pd.read_csv('data.csv', delimiter=',')
trainDF.head()

In [ ]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['category'],test_size=0.15,random_state=42)

In [ ]:
trainDF['category'].value_counts()

Employement Agreement     57
Restricted Stock Units    40
Incentive Plan            40
Compensation Agreement    20
Release and separation    18
Promissory Note           12
Service Agreement         12
Lease Agreement           11
Purchase Agreement        10
Loan Agreement             9
Credit Agreement           9
change of control          7
License Agreement          7
consulting agreement       7
Subscription Agreement     6
Name: category, dtype: int64

In [ ]:
# labelEncoder = preprocessing.LabelEncoder()
# labelEncoder.fit(['Employement Agreement','Restricted Stock Units','Incentive Plan','Compensation Agreement','Release and separation','Promissory Note','Service Agreement','Lease Agreement', 'Purchase Agreement', 'Loan Agreement','Credit Agreement', 'change of control','License Agreement', 'consulting agreement','Subscription Agreement'   ])
# #numericalLabels_train = labelEncoder.transform(train_y)
# #numericalLabels_valid = labelEncoder.transform(valid_y)

# encoder = preprocessing.LabelEncoder()
# trainLabels = encoder.fit_transform(train_y)
# trainLabels = [np_utils.to_categorical(i, num_classes=9) for i in trainLabels]
# trainLabels = np.asarray(trainLabels)
# print(trainLabels.shape)
# validLabels = encoder.fit_transform(valid_y)
# validLabels = [np_utils.to_categorical(i, num_classes=9) for i in validLabels]
# validLabels = np.asarray(validLabels)
# print(validLabels.shape)


labelEncoder = preprocessing.LabelEncoder()
#labelEncoder.fit(['PER','LOC','DAT','CAR','ORG','QUA','CUR','PRC', 'TIM'])
labelEncoder.fit(['Employement Agreement','Restricted Stock Units','Incentive Plan','Compensation Agreement','Release and separation','Promissory Note','Service Agreement','Lease Agreement', 'Purchase Agreement', 'Loan Agreement','Credit Agreement', 'change of control','License Agreement', 'consulting agreement','Subscription Agreement'])
numericalLabels_train = labelEncoder.transform(train_y)
numericalLabels_valid = labelEncoder.transform(valid_y)

encoder = preprocessing.LabelEncoder()
trainLabels = encoder.fit_transform(train_y)
trainLabels = [np_utils.to_categorical(i, num_classes=15) for i in trainLabels]
trainLabels = np.asarray(trainLabels)
print(trainLabels.shape)
validLabels = encoder.fit_transform(valid_y)
validLabels = [np_utils.to_categorical(i, num_classes=15) for i in validLabels]
validLabels = np.asarray(validLabels)
print(validLabels.shape)

(225, 15)
(40, 15)


In [ ]:
# Get the FastText Telugu word embeddigns
#!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.te.300.vec.gz
#!gunzip /content/cc.te.300.vec.gz
#!mv /content/cc.te.300.vec '/content/drive/MyDrive/Question Classifier Telugu/'

In [ ]:
# Using FastText pre trained telugu embeddings
embeddings_index = {}

import numpy as np

from keras.preprocessing import text, sequence
# Provide the embeddings for the particluar language
for i, line in enumerate(open('wiki-news-300d-1M.vec',encoding="utf-8")):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
    

In [ ]:
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index
#print(token.word_index)

In [ ]:
#print(word_index)
input_size = 150
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x),maxlen=input_size)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x),maxlen=input_size)
print(train_seq_x)
print(train_seq_x.shape)
print(valid_seq_x.shape)

[[   44   463   489 ...   344   141     5]
 [    1   535    17 ...    14    15  5543]
 [12411     5   753 ...   303   559     5]
 ...
 [    2     8     2 ...    14    15  1306]
 [    3    32  1674 ...  3754   787     5]
 [   20   152   399 ...     6    86   389]]
(225, 150)
(40, 150)


In [ ]:
embedding_matrix = np.zeros((len(word_index)+1, 300))

for word,i in word_index.items():
        embedding_vector = embeddings_index.get(word)    # checking that particular indexed word in telugu embedding .vec file
        if embedding_vector is not None:                 # if it is found in that .vec file  
            embedding_matrix[i] = embedding_vector       # store that vector of that word in .vec file saved to embedding matrix
               
print(embedding_matrix)  
print(embedding_matrix.shape)

[[ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.0897  0.016  -0.0571 ...  0.1559 -0.0254 -0.0259]
 [-0.0063 -0.0253 -0.0338 ...  0.1155  0.0073  0.0168]
 ...
 [-0.0111 -0.0381 -0.0108 ...  0.2289 -0.0134  0.0808]
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 [-0.0612 -0.0729 -0.0567 ... -0.2237 -0.0349  0.1003]]
(15064, 300)


In [ ]:
def train_model(classifier, feature_vector_train, category, feature_vector_valid):
    classifier.fit(feature_vector_train, category)
    predictions = classifier.predict(feature_vector_valid)
    acc = metrics.accuracy_score(predictions,valid_y)
    #f1Score = metrics.f1_score(predictions, valid_y, average='macro')
    #classificationReport = classification_report(predictions, valid_y)
    # fit the training dataset on the classifier
    return acc    

In [ ]:
import inspect
from tensorflow.keras.optimizers import Adam

## CNN

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((input_size, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.25)(embedding_layer)

    #Add the convolutional layer
    conv_layer = layers.Convolution1D(256, 3, activation="tanh")(embedding_layer)
    
    #Add the pooling layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)
    
    # Add the output Layers
    output_layer1 = layers.Dense(128, activation="tanh")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(15, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    #model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    #model.compile(loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
classifier = create_cnn()
is_neural_net = True
classifier.fit(train_seq_x, trainLabels, epochs=2)
    
    # predict the labels on validation dataset
predictions = classifier.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)

acc = metrics.accuracy_score(predictions1, validLabels1)
#f1Score = metrics.f1_score(predictions1, validLabels,average='macro')
#classificationReport = classification_report(predictions1, validLabels)
print(acc)
#print(f1Score)

#print(predictions1)

#if is_neural_net:
    #predictions = predictions.argmax(axis=-1)
    
#print(metrics.accuracy_score(predictions, valid_y))
#print(metrics.classification_report (valid_y, np.argmax(predictions, axis = 1)))

## RNN LSTM

In [ ]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((150, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(150)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(150, activation="tanh")(lstm_layer)
    #output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(15, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
    return model

In [ ]:
classifier = create_rnn_lstm()
is_neural_net=True  
classifier.fit(train_seq_x, trainLabels, epochs=10)
    
    # predict the labels on validation dataset
predictions = classifier.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)

acc = metrics.accuracy_score(predictions1, validLabels1)
#f1Score = metrics.f1_score(predictions1, validLabels,average='macro')
#classificationReport = classification_report(predictions1, validLabels)

print(acc)
#print(f1Score)


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 150, 300)          4519200   
_________________________________________________________________
spatial_dropout1d_5 (Spatial (None, 150, 300)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 150)               270600    
_________________________________________________________________
dense_10 (Dense)             (None, 150)               22650     
_________________________________________________________________
dense_11 (Dense)             (None, 15)                2265      
Total params: 4,814,715
Trainable params: 295,515
Non-trainable params: 4,519,200
___________________________________________

## Bidirectional RNN

In [ ]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((150, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(150, activation="tanh")(lstm_layer)
    #output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(15, activation="softmax")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
    return model


In [ ]:
classifier = create_bidirectional_rnn()
is_neural_net=True  
classifier.fit(train_seq_x, trainLabels, epochs=50)
    
    # predict the labels on validation dataset
predictions = classifier.predict(valid_seq_x)
predictions1 = predictions.argmax(axis=-1)
validLabels1 = validLabels.argmax(axis=-1)

acc = metrics.accuracy_score(predictions1, validLabels1)
f1Score = metrics.f1_score(predictions1, validLabels1,average='weighted')
classificationReport = classification_report(predictions1, validLabels1)

print(acc)
print(f1Score)
print(classificationReport)


Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 150, 300)          4519200   
_________________________________________________________________
spatial_dropout1d_7 (Spatial (None, 150, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               241200    
_________________________________________________________________
dense_14 (Dense)             (None, 150)               30150     
_________________________________________________________________
dense_15 (Dense)             (None, 15)                2265      
Total params: 4,792,815
Trainable params: 273,615
Non-trainable params: 4,519,200
___________________________________________